In [2]:
import pandas as pd
import pylab as p
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit, train_test_split
import scipy
import random
import time
from multiprocessing import Pool

In [3]:
core_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/CORE.csv')
game_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/GAMES.csv')
player_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PLAYERS.csv', index_col=[0])
team_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/TEAM.csv', index_col=[0])
plays_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/CORE.csv', index_col=[1])
pass_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PASS.csv', index_col=[0])
rush_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/RUSH.csv', index_col=[0])
pass_df = pass_df.join(plays_df)
rush_df = rush_df.join(plays_df)
tackle_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/TACKLES.csv', index_col=[0])
intercept_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/INTS.csv', index_col=[0])
fumble_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/FUMBLES.csv', index_col=[0])
tackle_df = tackle_df.join(plays_df)
intercept_df = intercept_df.join(plays_df)
fumble_df = fumble_df.join(plays_df)


#win_orig_df = win_orig_df.join(game_df)
offense_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/OFFENSE.csv', index_col=[2])
offense_df['PLAYER'] = offense_df.index
defense_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/DEFENSE.csv', index_col=[2])
defense_df['PLAYER'] = defense_df.index
players_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PLAYERS.csv', index_col=[0])
offense_df = offense_df.join(players_df['POS1'])
offense_df = offense_df.rename(columns={'POS1':'POS'})
defense_df = defense_df.join(players_df['POS1'])
defense_df = defense_df.rename(columns={'POS1':'POS'})

players_df['player_id'] = players_df.index
# offense_df = offense_df.join(players_df)
# defense_df = defense_df.join(players_df)


In [4]:
pass_groups = pass_df.groupby(['GID', 'PSR']) 
snp_list = []
for name, group in pass_groups:
    snp_list.append({"GID":name[0], "PLAYER":name[1], "PASSER_SNP":len(group)})
snp_df = pd.DataFrame(snp_list)
snp_df = snp_df.fillna(0)
offense_df = pd.merge(offense_df, snp_df, how='left', on=['GID', 'PLAYER'])

pass_groups = pass_df.groupby(['GID', 'TRG']) 
snp_list = []
for name, group in pass_groups:
    snp_list.append({"GID":name[0], "PLAYER":name[1], "TRG_SNP":len(group)})
snp_df = pd.DataFrame(snp_list)
snp_df = snp_df.fillna(0)
offense_df = pd.merge(offense_df, snp_df, how='left', on=['GID', 'PLAYER'])

rush_groups = rush_df.groupby(['GID', 'BC']) 
snp_list = []
for name, group in rush_groups:
    snp_list.append({"GID":name[0], "PLAYER":name[1], "BC_SNP":len(group)})
snp_df = pd.DataFrame(snp_list)
snp_df = snp_df.fillna(0)
offense_df = pd.merge(offense_df, snp_df, how='left', on=['GID', 'PLAYER'])

offense_df['SNP'] = offense_df['BC_SNP'] + offense_df['TRG_SNP'] +offense_df['PASSER_SNP']

In [5]:
tackle_groups = tackle_df.groupby(['GID', 'TCK']) 
snp_list = []
for name, group in tackle_groups:
    snp_list.append({"GID":name[0], "PLAYER":name[1], "TACKLE_SNP":len(group)})
snp_df = pd.DataFrame(snp_list)
snp_df = snp_df.fillna(0)
defense_df = pd.merge(defense_df, snp_df, how='left', on=['GID', 'PLAYER'])

intercept_groups = intercept_df.groupby(['GID', 'INT']) 
snp_list = []
for name, group in intercept_groups:
    snp_list.append({"GID":name[0], "PLAYER":name[1], "INTERCEPT_SNP":len(group)})
snp_df = pd.DataFrame(snp_list)
snp_df = snp_df.fillna(0)
defense_df = pd.merge(defense_df, snp_df, how='left', on=['GID', 'PLAYER'])

fumble_groups = fumble_df.groupby(['GID', 'FRCV']) 
snp_list = []
for name, group in fumble_groups:
    snp_list.append({"GID":name[0], "PLAYER":name[1], "FUMBLE_SNP":len(group)})
snp_df = pd.DataFrame(snp_list)
snp_df = snp_df.fillna(0)
defense_df = pd.merge(defense_df, snp_df, how='left', on=['GID', 'PLAYER'])

defense_df['SNP'] = defense_df['TACKLE_SNP'] + defense_df['INTERCEPT_SNP'] + defense_df['FUMBLE_SNP']

In [6]:
offense_df['GID_tmp'] = offense_df['GID']
offense_df['TEAM_tmp'] = offense_df['TEAM']
offense_df = offense_df.set_index(['GID', 'TEAM'])
offense_df['GID'] = offense_df['GID_tmp']
offense_df['TEAM'] = offense_df['TEAM_tmp']


defense_df['GID_tmp'] = defense_df['GID']
defense_df['TEAM_tmp'] = defense_df['TEAM']
defense_df = defense_df.set_index(['GID', 'TEAM'])
defense_df['GID'] = defense_df['GID_tmp']
defense_df['TEAM'] = defense_df['TEAM_tmp']

del offense_df['TEAM_tmp']
del offense_df['GID_tmp']
del defense_df['TEAM_tmp']
del defense_df['GID_tmp']



In [7]:
player_id_map = {}
for i, player_id in enumerate(player_df.index):
    player_id_map[player_id] = i
print player_id_map

{'CL-1700': 1194, 'JG-3400': 2910, 'BP-1100': 733, 'JH-1800': 2940, 'EP-0400': 2220, 'RA-0900': 5055, 'FW-0200': 2325, 'MP-2100': 4523, 'BD-0700': 531, 'SW-1500': 5855, 'DD-0450': 1573, 'AP-1800': 308, 'JB-8400': 2713, 'AP-0400': 294, 'KH-0500': 3658, 'LG-0800': 3967, 'JM-3400': 3160, 'AJ-1500': 219, 'CS-3100': 1343, 'KH-2900': 3682, 'JB-5400': 2680, 'TF-1325': 6011, 'MG-2000': 4291, 'MM-2700': 4452, 'JH-0600': 2926, 'MB-1200': 4150, 'JS-2000': 3344, 'AH-1600': 178, 'GC-0900': 2361, 'DP-2300': 1900, 'AH-0800': 170, 'JG-2200': 2895, 'FV-0100': 2323, 'MS-5500': 4616, 'BB-2560': 471, 'WM-0550': 6447, 'GS-0100': 2446, 'CB-1160': 888, 'XO-0100': 6501, 'SE-0100': 5618, 'OB-0250': 4825, 'JP-2800': 3259, 'TA-0300': 5871, 'WH-1100': 6431, 'MP-1100': 4513, 'JC-0500': 2722, 'FS-0300': 2316, 'JC-3700': 2756, 'CD-1200': 1001, 'ME-0500': 4252, 'DC-1700': 1534, 'AJ-0100': 201, 'SR-0600': 5777, 'JC-2250': 2740, 'TJ-2900': 6105, 'DB-4100': 1491, 'KW-1100': 3874, 'TW-2100': 6306, 'CS-0100': 1311, 'DS-36

In [8]:
ignore_columns = ['PLAYER', 'GID', 'TID', 'TNAME', 'POS', 'UID', 'TEAM']
categorical_to_binary_columns = ['V', 'H','STAD', 'WDIR', 'COND', 'SURF', 'TEAM', 'DAY']
convert_to_float_columns = ['HUMD', 'WSPD']


In [9]:
def get_game_features(index, game, my_team):
    feature = {}
    #setup the features from the game data (weather, point spread and stuff)
    if my_team == game['V'].values[0]:
        feature['is_visitor'] = 1.0
    else:
        feature['is_visitor'] = 0.0
    
    for key, val in game.iteritems():
        if key in ignore_columns:
            continue
        if key in categorical_to_binary_columns:
            if val.values[0] != "\N":
                feature['IS_{0}_{1}'.format(index, val.values[0])] = 1.0
        else:
            if str(val.dtype) == "object":
                if key in convert_to_float_columns:
                    if str.isdigit(val.values[0].strip()):
                        feature['{0}_{1}'.format(index, key)] = float(val.values[0])
                    else:
                        feature['{0}_{1}'.format(index, key)]  = 0.0
                else:
                    feature['{0}_{1}'.format(index, key)] = val.values[0]
            else:
                feature['{0}_{1}'.format(index, key)] = val.values[0]
    return feature

def get_team_game_features(index, team_game_data, game, players_team):
    feature = {}
    for team_game_row in team_game_data.iterrows():
        this_player_team = False
        # setup which team for this game is this_players team
        if players_team == team_game_row[1]['TNAME']:
            this_player_team = True
        if not this_player_team:
            continue
        # setup the features from the teams participating in the game
        for key, val in team_game_row[1].iteritems():
            if key in ignore_columns:
                continue
            if key in categorical_to_binary_columns:
                feature['IS_{0}_{1}_{2}'.format(index, val.values[0], 'my' if this_player_team else 'opp')] = 1.0
            else:
                feature['{0}_{1}_{2}'.format(index, 'my' if this_player_team else 'opp', key)] = val
    return feature

In [10]:
def get_features_for_player(index, offesne_player_row):
    feature = {}
    for key, val in offesne_player_row.iteritems():
        if key in ignore_columns:
            continue
        if key in categorical_to_binary_columns:
            feature["IS_{0}_{1}".format(val, index)] = 1.0
        else:
            feature["{0}_{1}".format(key, index)] = val
    return feature

In [11]:
def get_career_features_for_player(stats_df, player_id, game_id):
    feature = {}
    all_player = stats_df[(stats_df['PLAYER'] == player_id) & (stats_df['GID'] < game_id)]
#     all_player = stats_df[(stats_df['PLAYER'] == player_id)]
    for i, column in enumerate(all_player.columns):
        col_name = all_player.columns[i]
        if col_name in ignore_columns or str(all_player.dtypes[i]) == "object":
            continue
        feature["career_{0}_sum".format(col_name)] = all_player[column].sum()
        feature["career_{0}_mean".format(col_name)] = all_player[column].mean()
        feature["career_{0}_std".format(col_name)] = all_player[column].std()
    return feature

In [12]:
def get_moving_mean_features(index, moving_mean):
    feature = {}
    for i, col_name in enumerate(moving_mean.columns):
        if col_name in ignore_columns:
            continue
        if col_name not in categorical_to_binary_columns:
            feature["moving_average_{0}_{1}_sum".format(index, col_name)] = moving_mean[col_name].sum()
            feature["moving_average_{0}_{1}_mean".format(index, col_name)] = moving_mean[col_name].mean()
            feature["moving_average_{0}_{1}_std".format(index, col_name)] = moving_mean[col_name].std()
    return feature

In [13]:
def get_okay_cols(df, ignore_categorical=True): 
    okay_cols = []
    for col in df.columns:
        if (col not in ignore_columns and (not ignore_categorical or col not in categorical_to_binary_columns)):
            okay_cols.append(col)
    return okay_cols

def get_flattened_columns(prefix, pivot_table):    
    cols = []
    col_dict = {}
    for i, (col, gid) in enumerate(pivot_table.columns.tolist()):
        if gid not in col_dict:
            col_dict[gid] = str(i)
        cols.append(prefix+col_dict[gid]+"_"+col)
    return cols


def get_historical_features_for_player(stats_df, player_id, game_id, my_team, opp_team, position):
    feature = {}
    target = None
    # all games < than the game in question
    player_games = stats_df[(stats_df['PLAYER'] == player_id) & (stats_df['GID'] <= game_id)]
    # resort the plays in game_id order
    player_games = player_games.sort(['GID'], ascending=[0])
    if len(player_games) > 0:
        # target is the game_id games value of Fantasy points
        target = player_games['FPTS'].values[0]
        # put in the career features
        feature['player_id'] = player_id_map[player_id]
        feature['is_{0}'.format(position)] = True
        feature.update(get_career_features_for_player(stats_df, player_id, game_id))
        # we want to use all the game data after the game_id
        player_games = player_games[1:5]
        okay_cols = get_okay_cols(player_games, False)
        okay_cols += ['PLAYER', 'GID']
        player_games = player_games[okay_cols]
        player_games_pivot = player_games.pivot(index='PLAYER', columns='GID')
        cols = get_flattened_columns("", player_games_pivot)
        player_games_pivot.columns = cols
        

        okay_cols = get_okay_cols(player_games)
        # expanding mean for games 1 - 4 after game_id
        expanding_mean = pd.expanding_mean(player_games[okay_cols][1:5], min_periods=1)
        expanding_mean['PLAYER'] = player_games['PLAYER']
        expanding_mean['GID'] = player_games['GID']
        expanding_mean_pivot = expanding_mean.pivot(index='PLAYER', columns='GID')
        cols = get_flattened_columns("expanding_mean_", expanding_mean_pivot)
        expanding_mean_pivot.columns = cols
        player_games_pivot = player_games_pivot.join(expanding_mean_pivot)
        
        # expanding std for games 1 - 4 after game_id
        expanding_mean = pd.expanding_std(player_games[okay_cols][1:5], min_periods=1)
        expanding_mean['PLAYER'] = player_games['PLAYER']
        expanding_mean['GID'] = player_games['GID']
        expanding_mean_pivot = expanding_mean.pivot(index='PLAYER', columns='GID')
        cols = get_flattened_columns("expanding_std_", expanding_mean_pivot)
        expanding_mean_pivot.columns = cols
        player_games_pivot = player_games_pivot.join(expanding_mean_pivot)
        
        # expanding sum for games 1 - 4 after game_id
        expanding_mean = pd.expanding_sum(player_games[okay_cols][1:5], min_periods=1)
        expanding_mean['PLAYER'] = player_games['PLAYER']
        expanding_mean['GID'] = player_games['GID']
        expanding_mean_pivot = expanding_mean.pivot(index='PLAYER', columns='GID')
        cols = get_flattened_columns("expanding_sum_", expanding_mean_pivot)
        expanding_mean_pivot.columns = cols
        player_games_pivot = player_games_pivot.join(expanding_mean_pivot)
        for i, (index, row) in enumerate(player_games_pivot.iterrows()):
            feature = {}
            for key, val in row.iteritems():
                if key in ignore_columns:
                    continue
                if key in categorical_to_binary_columns:
                    feature["IS_{0}_{1}".format(val, index)] = 1.0
                else:
                    feature["{0}_{1}".format(key, index)] = val
#         print player_plays_pivot.columns
        
#         for i,(index, row) in enumerate(player_plays.iterrows()):
#             player_id = row['PLAYER']
#             # target is this games fantasy points
            
#             pos = row['POS']
#             # features are all the older games not including this one [i:] historical data
#             # this means there is a feature for every game for each player
#             moving_mean = pd.DataFrame(columns=player_plays.columns)
#             for k, (index, player_row) in enumerate(player_plays[i+1:(i+5)].iterrows()):
#                 game_id = player_row['GID']
#                 # this will be the 2 teams that played
#                 team_game_data = team_df[team_df['GID'] == game_id]
#                 # this is basic info on the game, who is home and visitor
#                 game = game_df[game_df['GID'] == game_id]
# #                 feature.update(get_game_features(k, game, my_team))
# #                 feature.update(get_team_game_features(k, team_game_data, game, my_team))
#                 # setup the features for the player
#                 feature.update(get_features_for_player(k, player_row))
#                 moving_mean.loc[k] = player_row
#                 feature.update(get_moving_mean_features(k, moving_mean))
    return feature, target

In [14]:
import datetime
def get_features_for_game(game_id, team, opp_team):
    game_features = []
#     print game_id, team, opp_team
#     start = datetime.datetime.now()
    offense_game_players = offense_df.ix[game_id].ix[team]
#     print (datetime.datetime.now()-start).total_seconds()   
#     offense_game_players = offense_df[(offense_df['GID'] == game_id) & (offense_df['TEAM'] == team)]
    defense_game_players = defense_df.ix[game_id].ix[opp_team]
#     defense_game_players = defense_df[(defense_df['GID'] == game_id) & (defense_df['TEAM'] == opp_team)]

    print "game_id:", game_id, "num_players:", len(offense_game_players), len(defense_game_players)
    # so everything is ordered the same, order by position and then by snaps descending, so the
    # position with the most snaps is always first
    offense_game_players = offense_game_players.sort(['POS', 'SNP'], ascending=[1, 0])
    defense_game_players = defense_game_players.sort(['POS', 'SNP'], ascending=[1, 0])
    offense_game_players_features = {}
    defense_game_players_features = {}

    # offense players
#     start = datetime.datetime.now()
    for j, (index, current_player) in enumerate(offense_game_players.iterrows()):
        current_player_id = current_player['PLAYER']
        # we go through the "team" players as offense
        historical_features, target = get_historical_features_for_player(offense_df, current_player_id, game_id, team, opp_team, current_player['POS'])
        offense_game_players_features[current_player_id] = (historical_features, target, current_player['POS'])
#     print (datetime.datetime.now()-start).total_seconds()   

    #defensive players
#     start = datetime.datetime.now()
    for j, (index, current_player) in enumerate(defense_game_players.iterrows()):        
        current_player_id = current_player['PLAYER']
        # and the "opp_team" is the defense
        historical_features, target = get_historical_features_for_player(defense_df, current_player_id, game_id, opp_team, team, current_player['POS'])
        defense_game_players_features[current_player_id] = (historical_features, target, current_player['POS'])
#     print (datetime.datetime.now()-start).total_seconds()   
#     print "finished getting features for players for game ", game_id 
    start = datetime.datetime.now()
    for k, (player_id_focus, (features_focus, target, pos_focus)) in enumerate(offense_game_players_features.iteritems()):
        feature = {'target':target}  
        for j, (player_id, (features, target, pos)) in enumerate(offense_game_players_features.iteritems()):
            prefix = "off_{0}_{1}".format(pos, j)
            # if we are working on the current player as the focus then assign him the current tag
            if player_id_focus == player_id:
                prefix = "off_current"
            else:
                continue
            for i, (key, val) in enumerate(features.iteritems()):
                feature['{0}_{1}'.format(prefix, key)] = val
#         for j, (player_id, (features, target, pos)) in enumerate(defense_game_players_features.iteritems()):
#             prefix = "def_{0}_{1}".format(pos, j)
#             for i, (key, val) in enumerate(features.iteritems()):
#                 feature['{0}_{1}'.format(prefix, key)] = val

        game_features.append(feature)
#     print (datetime.datetime.now()-start).total_seconds()   

    return game_features

In [15]:
def get_features_for_game_proxy(args):
    game_id, team, opp_team, game_count = args
    results = []
    try:
        results = get_features_for_game(game_id, team, opp_team)
    except Exception as e:
        print "exception",e
#     if game_count%10 == 0:
#         print "finished up to ", game_id
    return results

In [ ]:
total_count = len(game_df.index)
all_features = []
game_df = game_df.sort(['GID'], ascending=[0])
cpu_count = 7
pool = Pool(processes=cpu_count)
pool_queue = []
for game_count, (game_index, game) in enumerate(game_df.iterrows()):
    game_id = game['GID']
    teams = [game['H'], game['V']]
    for team in teams:
        opp_team = teams[0]
        if team == teams[0]:
            opp_team = teams[1]
        pool_queue.append([game_id, team, opp_team, game_count])
#         print get_features_for_game(game_id, team, opp_team)        
#         all_features += get_features_for_game(game_id, team, opp_team)
#         break
#     break
    if game_count > 500:
        break
results = pool.map(get_features_for_game_proxy, pool_queue, 1)
for result in results:
    all_features += result
        

game_id: 3455 num_players: 8 17
game_id: 3452 num_players: 8 16
game_id: 3455 num_players: 11 14
game_id: 3454 num_players: 8 15
game_id: 3453 num_players: 9 14
game_id: 3454 num_players: 10 15
game_id: 3453 num_players: 9 13
game_id:game_id:game_id:game_id:game_id:game_id:game_id: 3449 num_players: 9 15
 3449 num_players: 10 15
 3450 num_players: 10 14
 3451 num_players: 9 17
 3451 num_players: 10 15
 3450 num_players: 9 13
 3452 num_players: 9 16
game_id:game_id:game_id:game_id:game_id:game_id:game_id: 3446 num_players: 11 17
 3445 num_players: 10 18
 3447 num_players: 9 17
 3446 num_players: 9 16
 3447 num_players: 10 14
 3448 num_players: 9 17
 3448 num_players: 10 15
game_id:game_id:game_id:game_id:game_id:game_id:game_id: 3442 num_players: 10 15
 3442 num_players: 10 18
 3443 num_players: 10 16
 3443 num_players: 12 16
 3444 num_players: 9 18
 3444 num_players: 11 17
 3445 num_players: 7 13
game_id:game_id:game_id:game_id:game_id:game_id:game_id: 3439 num_players: 9 14
 3438 num_

In [ ]:
features_df = pd.DataFrame(all_features)
features_df = features_df.fillna(0.0)
print features_df
# for i, col in enumerate(features_df.columns):
#     print features_df.columns[i], features_df.dtypes[i]

In [1]:
features_df = features_df.fillna(0.0)
if 'target' in features_df.columns:
    targets = features_df['target']
if 'target' in features_df.columns:
    del features_df['target']
x_train, x_test, y_train, y_test = train_test_split(features_df, targets, test_size=0.33, random_state=42)
cfr = ExtraTreesRegressor(
    oob_score = True, bootstrap=True, verbose = 1, max_features = None, n_estimators = 100, min_samples_leaf = 1,
    n_jobs=7,
    random_state=0,
)
cfr.fit(x_train, y_train)



NameError: name 'features_df' is not defined

In [ ]:
score = cfr.score(x_test, y_test)
print score

In [ ]:
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), cfr.feature_importances_), x_train.columns), 
             reverse=True)

In [ ]:
predictions = cfr.predict(x_test)
plt.scatter(y_test, predictions)
plt.show()